In [1]:
import cmath
from copy import copy, deepcopy
import itertools
import warnings

from complex import *
from ComplexVector import *
from FuncClasses import *
from RegionClasses import *
from regions import *
from utils import *

# import cvxpy as cp
import numpy as np
import sympy as sym
from z3 import *
import scipy as sp
from scipy.optimize import NonlinearConstraint, Bounds, minimize, linprog, OptimizeWarning

# Functions

## Z3

In [2]:
def z3_find_b(barrier, states, f_vec, term_powers, coeffs, prec=2):
    dbdz = FuncVec([diff(barrier, i) for i in range(states)])
    dbdzconj = FuncVec([diff(barrier, i) for i in range(states, 2*states)])
    dbdt = (dbdz * f_vec) + (dbdzconj * f_vec.conj())
    print("Differential of barrier made...")
    
    barr_minus_conj = barrier - barrier.conj()
    
    s = Solver()
    # Basic conditions
    s.add([simplify(coeff == 0) for coeff in dbdt.get_coeffs()])
    s.add([simplify(coeff == 0) for coeff in barr_minus_conj.get_coeffs()])
    # s.add(Not(And([coeff == 0 for coeff in barrier.get_coeffs()])))

    idx = []
    for i in range(states):
        temp = [0]*states
        temp[i] = 1
        idx.append(term_powers.index(temp*2))
    prob_coeffs = [coeffs[i] for i in idx]
    s.add(sum(prob_coeffs) == 1)
    # Make coeffs of probability sum distinct (a_i from sum(a_i z_i*conj(z_i)))
    d = And([prob_coeffs[i] == prob_coeffs[i+1] for i in range(len(prob_coeffs) - 1)])
    s.add(Or(Not(d), And([And(p.r == 0,p.i==0) for p in prob_coeffs])))

    print("Solver ready...")
    ms = print_all_models(s)
    return barrier.get_sym_sum(ms[0], prec)


In [3]:
def z3_find_constant(barrier, init_conditions=[]):
    zs = barrier.free_symbols
    z3_vars, z3_barrier = sympy_to_z3(zs, barrier)
    z0 = Complex('z0')
    z1 = Complex('z1')
    o = Optimize()
    o.add(z0.len_sqr() >= 0.9, z0.len_sqr() <= 1)
    o.add(z1.len_sqr() <= 0.1, z1.len_sqr() >= 0)
    o.add(z1.len_sqr() + z0.len_sqr() == 1)
    o.maximize(z3_barrier.r)
    o.set(timeout=5000)
    # c = Real('c')
    # s = Solver()
    # init = And(z0.len_sqr() >= 0.9, z0.len_sqr() <= 1, z1.len_sqr() <= 0.1, z1.len_sqr() >= 0, z0.len_sqr() + z1.len_sqr() == 1)
    # s.add(ForAll([z0.r,z0.i,z1.r,z1.i], Implies(init, c + z3_barrier.r >= 0)))
    print("Optimizing")
    print(o.check())
    model = o.model()
    
    print("Getting value...")
    vs = [(v, get_real_from_model(model, v.r) + 1j*get_real_from_model(model, v.i)) for v in z3_vars]
    value = barrier
    # Pair symbols with values from Z3
    for sym in barrier.free_symbols:
        for v in vs:
            if str(sym) == str(v[0]): value = value.subs({sym: v[1]})
    return -value

## Scipy

In [4]:
def diff_fsum(fsum, i):
    new_fsum = []
    for fterm in fsum.fterms:
        t = deepcopy(fterm.var)
        if t[i] != 0:
            c = fterm.coeff * t[i]
            t[i] -= 1
            new_fsum.append(FuncTerm(c, t))
    return FuncSum(new_fsum)

def scipy_find_b(barrier, states, f_vec, term_powers, prec=2, linprog_obj=[]):
    # Make appropriate conditions using representation
    dbdz = FuncVec([diff_fsum(barrier, i) for i in range(states)])
    dbdzconj = FuncVec([diff(barrier, i) for i in range(states, 2*states)])
    dbdt = (dbdz * f_vec) + (dbdzconj * f_vec.conj())
    barr_minus_conj = barrier - barrier.conj()
    
    # Create linprog matrices and vectors
    A_dbdt = [list(term.coeff.imag) for term in dbdt.fterms]
    b_dbdt = [0]*len(A_dbdt)
    
    A_conj = [list(term.coeff) for term in barr_minus_conj.fterms]
    b_conj = [0]*len(A_conj)
    
    A = np.array(A_dbdt + A_conj)
    b = np.array(b_dbdt + b_conj)
    # Minimize vector (min c^T x subject to ...)
    c = np.array([0]*len(term_powers))
    for i in linprog_obj:
        c[i] = 1
    
    # Negate A doesn't matter unless used for A_ub
    res = linprog(c,
                  A_eq=A,
                  b_eq=b,
                  bounds=(-1,1),
                 )
    return barrier.get_sym_sum(res.x, prec)

In [5]:
def scipy_find_constant(barrier_sym, states, init=[], prec=2):
    # Create objective function from Sympy barrier
    def obj(x):
        z = [x[2*i] + 1j*x[2*i + 1] for i in range(states)]
        b = barrier_sym
        for var_sym in barrier_sym.free_symbols:
            sym_num = int(str(var_sym)[1:])
            b = b.subs({var_sym: z[sym_num]})
        return -float(sym.re(b))

    # Bounds and guesses
    bounds = Bounds([-1]*2*states, [1]*2*states)
    guess = [0]*2*states
    guess[0] = 1
    
    res = minimize(obj,
                   guess,
                   method='trust-constr',
                   constraints=init,
                   options={'verbose': 0},
                   bounds=bounds,
                   hess=lambda x: np.zeros((2*states,))
                  )
    minimum = round(res.fun, prec)
    return minimum

In [6]:
def scipy_check_constant(c, barrier_sym, states, unsafe=[], prec=2):
    def obj(x):
        z = [x[2*i] + 1j*x[2*i + 1] for i in range(states)]
        b = barrier_sym
        for var_sym in barrier_sym.free_symbols:
            sym_num = int(str(var_sym)[1:])
            b = b.subs({var_sym: z[sym_num]})
        return float(sym.re(b))

    # Bounds and guesses
    bounds = Bounds([-1]*2*states, [1]*2*states)
    guess = [0]*2*states
    
    res = minimize(obj,
                   guess,
                   method='trust-constr',
                   constraints=unsafe,
                   options={'verbose': 0},
                   bounds=bounds,
                   hess=lambda x: np.zeros((2*states,))
                  )
    minimum = round(res.fun, prec)
    if -minimum >= c : raise Exception("Error: barrier has part of unsafe in same contour as initial region")

In [7]:
# Setup coefficients as ndarray
def scipy_find_k_barrier(k, H, init=[], unsafe=[], linprog_obj = [], prec=2, verbose=False):
    z = -1j
    n = round(len(H))
    term_powers = generate_term_powers(k, n)
    coeff_num = len(term_powers)
    

    if verbose: print("Converting dynamical system...")
    sums = []
    for i in range(n):
        terms = []
        for j in range(n):
            t = [0]*(2*n)
            t[j] = 1
            t = tuple(t)
            terms.append(FuncTerm(z * H[i][j], t))
        sums.append(FuncSum(terms))
    f_vec = FuncVec(sums)
    if verbose: print("Dynamical system converted.")
    
    id_coeff = np.identity(coeff_num)
    barrier = FuncSum(list([FuncTerm(i, t) for i, t in zip(id_coeff, term_powers)]))
    if verbose: print("Finding polynomial...")
    warnings.simplefilter("ignore", OptimizeWarning)
    b = scipy_find_b(barrier, n, f_vec, term_powers, prec, linprog_obj)
    if verbose: print("Polynomial found: ", b)
    
    if verbose: print("Finding constant...")
    c = scipy_find_constant(b, n, init=init)
    if verbose: print("Checking...")
    scipy_check_constant(c, b, n, unsafe=unsafe)
    if verbose: print("Constant found: ", c)
    
    return round_sympy_expr(c + b)

## General Method

In [8]:
def find_b(barrier, states, f_vec, term_powers, coeffs, prec=2, package="scipy"):
    if package == "z3": return z3_find_b(barrier, states, f_vec, term_powers, coeffs, prec)
    if package == "scipy": return scipy_find_b(barrier, states, f_vec, term_powers, coeffs, prec)

def find_k_barrier(k, H, constraints=[], prec=2, package="scipy"):
    z = 0-I
    n = round(len(H))
    print("Converting dynamical system...")
    sums = []
    for i in range(n):
        terms = []
        for j in range(n):
            t = [0]*(2*n)
            t[j] = 1
            t = tuple(t)
            terms.append(FuncTerm(z * H[i][j], t))
        sums.append(FuncSum(terms))
    f_vec = FuncVec(sums)
    print("Dynamical system converted.")
    
    term_powers = generate_term_powers(k, n)
    c0 = Complex('c')
    coeffs = [c0] + ComplexVector('a', len(term_powers) - 1)
    barrier = FuncSum(list([FuncTerm(c, t) for c, t in zip(coeffs[1:], term_powers[1:])]))
    print("Finding polynomial...")
    b = find_b(barrier, n, f_vec, term_powers, coeffs, prec, package=package)
    print("Finding constant...")
    c = scipy_find_constant(b, n, constraints=constraints)
    return c + b

# Using Z3

In [9]:
# Hadamard as a Hamiltonian
H = [[1/np.sqrt(2), 1/np.sqrt(2)],[1/np.sqrt(2), -1/np.sqrt(2)]]
# H = [[1,1],[1,-1]]
def f(x): return [x[0]**2 + x[1]**2, x[2]**2 + x[3]**2, x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2]
constraints = [NonlinearConstraint(f, [0.9, 0, 1], [1,0.1, 1])]
# b = find_k_barrier(2, H, constraints=constraints, package="z3")
# b

In [10]:
# TODO: Try out new Hamiltonians and regions
H = [[0,0,0,0],[0,0,0,0],[0,0,np.pi/np.sqrt(2),-np.pi/np.sqrt(2)],[0,0,-np.pi/np.sqrt(2),np.pi/np.sqrt(2)]]
# init = InitRegion(near_00)
# unsafe = UnsafeRegion(near_11)
def f(x): return [x[0]**2 + x[1]**2, x[2]**2 + x[3]**2, x[4]**2 + x[5]**2, x[6]**2 + x[7]**2, x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2 + x[4]**2 + x[5]**2 + x[6]**2 + x[7]**2]
constraints = [NonlinearConstraint(f, [0.81, 0, 0, 0, 1], [1, 0.09, 0.09, 0.1, 1])]

# init = near_10
# unsafe = in_0region
def f(x): return [x[0]**2 + x[1]**2, x[2]**2 + x[3]**2, x[4]**2 + x[5]**2, x[6]**2 + x[7]**2, x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2 + x[4]**2 + x[5]**2 + x[6]**2 + x[7]**2]
constraints = [NonlinearConstraint(f, [0, 0, 0.81, 0, 1], [0.09, 0.01, 1, 0.09, 1])]

# find_k_barrier(2, H, constraints=constraints, package="z3")

# Using Scipy

In [11]:
# Hadamard as a Hamiltonian
# H = [[1/np.sqrt(2), 1/np.sqrt(2)],[1/np.sqrt(2), -1/np.sqrt(2)]]
H = [[1,1],[1,-1]]
# For Had-like Hadamard
# c[12] = 1

def init(x): return [x[0]**2 + x[1]**2,
                     x[2]**2 + x[3]**2,
                     x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2]
init_constraints = [NonlinearConstraint(init, [0.9, 0, 1], [1, 0.1, 1])]

def unsafe(x): return [x[0]**2 + x[1]**2,
                       x[2]**2 + x[3]**2,
                       x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2]
unsafe_constraints = [NonlinearConstraint(init, [0, 0.9, 1], [0.1, 1, 1])]


barrier = scipy_find_k_barrier(2, H,
                               init=init_constraints,
                               unsafe=unsafe_constraints,
                               linprog_obj=[0,12],
                               verbose=1)
barrier

Converting dynamical system...
Dynamical system converted.
Finding polynomial...
Polynomial found:  -1.0*z0*conjugate(z0) - 0.33*z0*conjugate(z1) - 0.33*z1*conjugate(z0) - 0.33*z1*conjugate(z1) - 1.0
Finding constant...
Checking...
Constant found:  1.74


-1.0*z0*conjugate(z0) - 0.33*z0*conjugate(z1) - 0.33*z1*conjugate(z0) - 0.33*z1*conjugate(z1) + 0.74

3 * Barrier = $ \displaystyle - 3.0 z_{0} \overline{z_{0}} - 1.0 z_{0} \overline{z_{1}} - 1.0 z_{1} \overline{z_{0}} - 1.0 z_{1} \overline{z_{1}} + 2.2$


In Bra-ket notation: $ \displaystyle -3\langle 0| \phi \rangle \langle \phi| 0\rangle - 1 \langle 0| \phi\rangle \langle \phi|1 \rangle -1 \langle 1|\phi \rangle \langle \phi|0 \rangle - 1 \langle 1|\phi \rangle \langle \phi| 1\rangle + 2.2$

In [12]:
# init = InitRegion(near_00)
# unsafe = UnsafeRegion(near_11)
# def f(x): return [x[0]**2 + x[1]**2, x[2]**2 + x[3]**2, x[4]**2 + x[5]**2, x[6]**2 + x[7]**2, x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2 + x[4]**2 + x[5]**2 + x[6]**2 + x[7]**2]
# constraints = [NonlinearConstraint(f, [0.81, 0, 0, 0, 1], [1, 0.09, 0.09, 0.01, 1])]

In [13]:
H = [[0,0,0,0],[0,0,0,0],[0,0,np.pi/np.sqrt(2),-np.pi/np.sqrt(2)],[0,0,-np.pi/np.sqrt(2),np.pi/np.sqrt(2)]]
# H = [[0,0,0,0],[0,0,0,0],[0,0,1,-1],[0,0,-1,1]]
# For CNOT
# c[16] = 1
# c[23] = 1
# c[-14] = 1
# c[-5] = 1

# init = near_10
# unsafe = in_0region
def init(x): return [x[0]**2 + x[1]**2,
                     x[2]**2 + x[3]**2,
                     x[4]**2 + x[5]**2,
                     x[6]**2 + x[7]**2,
                     x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2 + x[4]**2 + x[5]**2 + x[6]**2 + x[7]**2]
init_constraints = [NonlinearConstraint(init, [0, 0, 0.81, 0, 1], [0.09, 0.01, 1, 0.09, 1])]

def unsafe(x): return [x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2,
                     x[4]**2 + x[5]**2 + x[6]**2 + x[7]**2,
                     x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2 + x[4]**2 + x[5]**2 + x[6]**2 + x[7]**2]
unsafe_constraints= [NonlinearConstraint(unsafe, [.5,0,1], [1,.5,1])]

b_cnot = scipy_find_k_barrier(2, H,
                              init=init_constraints,
                              unsafe=unsafe_constraints,
                              linprog_obj=[0,16,23],
                              verbose=1)
b_cnot

Converting dynamical system...
Dynamical system converted.
Finding polynomial...
Polynomial found:  -1.0*z2*conjugate(z2) - 1.0*z3*conjugate(z3) - 1.0
Finding constant...
Checking...
Constant found:  1.9


-1.0*z2*conjugate(z2) - 1.0*z3*conjugate(z3) + 0.9

Barrier found:
$
\displaystyle - 1.0 z_{2} \overline{z_{2}} - 1.0 z_{3} \overline{z_{3}} + 0.9
$

In [14]:
d = 0
H = [[0,0,d,-d],
     [0,0,d,-d],
     [d,d,1,-1],
     [-d,-d,-1,1]]


# init = near_10
# unsafe = in_0region
def init(x): return [x[0]**2 + x[1]**2,
                     x[2]**2 + x[3]**2,
                     x[4]**2 + x[5]**2,
                     x[6]**2 + x[7]**2,
                     x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2 + x[4]**2 + x[5]**2 + x[6]**2 + x[7]**2]
init_constraints = [NonlinearConstraint(init, [0, 0, 0.81, 0, 1], [0.09, 0.01, 1, 0.09, 1])]

def unsafe(x): return [x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2,
                     x[4]**2 + x[5]**2 + x[6]**2 + x[7]**2,
                     x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2 + x[4]**2 + x[5]**2 + x[6]**2 + x[7]**2]
unsafe_constraints= [NonlinearConstraint(unsafe, [.5,0,1], [1,.5,1])]

# b_ctop = scipy_find_k_barrier(2, H,
#                               init=init_constraints,
#                               unsafe=unsafe_constraints,
#                               linprog_obj=[0,16,23],
#                               verbose=1)
# b_ctop